#Лабораторная работа 7

In [107]:
 from sklearn.datasets import load_digits
 mnist = load_digits()
 X = mnist['images']
 X = X.reshape((X.shape[0], -1))
 y = mnist['target']
 X.shape

(1797, 64)

In [108]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

##Задание 1

###Задание 1.1

In [109]:
import numpy as np

Training

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import time

start = time.time()


classifiers = []
for i in range(10):
    y_i = np.where(y_train == i, 1, 0)
    # X_train, X_test, y_train, y_test = train_test_split(X, y_i, test_size=0.3, random_state=42)
    cls = LogisticRegression(random_state=6, max_iter=1000)
    cls.fit(X_train, y_i)
    classifiers.append(cls)

end = time.time()

print(f'Training time: {end - start} s')


Training time: 2.1158971786499023 s


In [111]:
probs = np.zeros((len(y_test), 10))

for i in range(10):
    probs[:, i] = classifiers[i].predict_proba(X_test)[:,1]

In [112]:
y_pred = np.argmax(probs, axis=-1)

In [113]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9592592592592593

###Задание 1.2

In [114]:
start = time.time()

classifiers_one_vs_one = []
for i in range(10):
    cls_i = []
    for j in range(i+1, 10):
        mask_i_j = np.array(y_train == i) | np.array(y_train == j)


        X_i_j = X_train[mask_i_j]
        y_i_j = y_train[mask_i_j]
  
        y_i_j = np.where(y_i_j == i, 1, 0)

        assert len(y_i_j) == len(X_i_j) 
       

        cls_i_j = LogisticRegression(random_state=i*j, max_iter=1000)
        cls_i_j.fit(X_i_j, y_i_j)

        cls_i.append(cls_i_j)
    classifiers_one_vs_one.append(cls_i)



end = time.time()

print(f'Training time: {end - start} s')

Training time: 1.5068109035491943 s


In [115]:
probs = np.zeros((len(y_test), 10, 10))

for i in range(10):
    for j in range(i + 1, 10):
        cls_i_j = classifiers_one_vs_one[i][j - i - 1]
        probs_pred =  cls_i_j.predict_proba(X_test)
        probs[:, i, j] = probs_pred[:, 1]
        probs[:, j, i] = probs_pred[:, 0]
        '''print(i,j)
        print(probs[:, i, j])'''

y_pred_one_vs_one = np.argmax(probs.sum(axis=2), axis=1)

In [116]:
accuracy_score(y_pred_one_vs_one, y_test)

0.987037037037037

###Отчёт

* Исходный датасет не модифицировался (помимо преобразования объектов из двумерных массивов в одномерные).

* Данные были разбиты на тренировочную и тестовую выборки в соотношении 7:3.

* В качестве метрики для оценки качества была выбрана accuracy.

* One vs One даёт лучшее качествою.

* Сложность One vs Rest - $O(k * n) = O(kn)$, One vs One - $O(k^2 * n/k) = O(k n)$, где $k$ - количество классов, $n$ - количество объектов. 

* One vs One на практике обучился быстрее. Это связано с тем, что при One vs Rest для обучения на одной итерации использовалась вся выборка, а для One vs One - где-то 20%.

* One vs Rest:

  $+$ Обучает меньше классификаторов

  $*$ Стоит использовать, когда классы не сбалансированы и когда много данных.

  One vs One:

  $+$ Классификаторы обучаются на сбалансированных выборках (если классы сбалансированны)

   $+$ На практике обучается быстрее (если классы сбалансированны)

   $*$ Стоит использовать, когда классы сбалансированы и/или когда мало данных.




##Задание 2

Выразим $\hat{y}$

$\hat{y} = argmax_{y' \in Y}(\Delta(y', y) + < w^{(+)}, \psi (x,y') - \psi(x,y)) = [\psi(x,y) = \sum_{i=1}^r \Phi(x, y_i, y_{i-1}), \Delta(y', y) = \sum_{i=1}^r \delta(y_i, y_{i-1})] = argmax_{y' \in Y}(\sum_{i=1}^r (\delta(y_i, y_{i-1}) + < w, \Phi(x, y'_i, y'_{i-1}) - \Phi(x, y_i, y_{i-1}) >)) = [ \Phi(x, y_i, y_{i-1}) = const, w=const  ] = argmax_{y' \in Y}(\sum_{i=1}^r (\delta(y_i, y_{i-1}) + < w, \Phi(x, y'_i, y'_{i-1})>))$

Определим функцию:

$F_{t, s} = \max_{(y'_1, \dots , y'_{t-1}, y'_t = s)} (\sum_{i=1}^t (\delta(y'_i, y_i) + < w, \Phi (x, y'_i, y'_{i-1}) >))$

Тогда можно записать следующее рекурсивное выражение:

$F_{t, s} = \max_{(y'_1, \dots , y'_{t-1}, y'_t = s)}  (F_{t - 1, y'_{t-1}} + \delta(s,y_t) + < w, \Phi (x, s, y'_{t-1}) >)$


##Задание 3

In [152]:
from sklearn.svm import LinearSVC

start = time.time()

svm = LinearSVC(random_state=42,  max_iter=300000)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

end = time.time()

print(f'Training time: {end - start} s')

Training time: 7.9218738079071045 s


In [153]:
y_pred_train = svm.predict(X_train)
accuracy_score(y_pred_train, y_train)

0.9984089101034208

In [154]:
accuracy_score(y_pred, y_test)

0.9351851851851852

###Отчёт

* Точность на тренировочной выборке - 0.9984, а на тестовой - 0.9352. Можно предположить, что алгоритм переобучился из-за большого количества итераций, но нет. Если уменьшать max_iter, качество на тестовой выборке ещё ниже.

* По сравнению с алгоритмами из Задания 1 SVM обучался гораздо дольше.